In [1]:
import numpy as np
import time
from datetime import timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Adjust the final dense layer to match the desired output shape (64, 64, 3)
    model.add(layers.Dense(64 * 64 * 3, activation='tanh'))
    model.add(layers.Reshape((64, 64, 3)))  # Reshape to (64, 64, 3)

    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Flatten())
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

In [3]:
cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [4]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.random.uniform(tf.shape(real_output), 0.9, 1.0), real_output)
    fake_loss = cross_entropy(tf.random.uniform(tf.shape(fake_output), 0.0, 0.1), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [5]:
def train_step(images):
    noise = tf.random.normal([batch_size, 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer = tf.keras.optimizers.Adam(tf.keras.optimizers.schedules.ExponentialDecay(1e-4, decay_steps=1000, decay_rate=0.9))
    discriminator_optimizer = tf.keras.optimizers.Adam(tf.keras.optimizers.schedules.ExponentialDecay(1e-4, decay_steps=1000, decay_rate=0.9))


In [6]:
def generate_images(model, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow((predictions[i, :, :] + 1) / 2.0)  # Scale to [0, 1]
        plt.axis('off')

    plt.show()

In [7]:
image_size = (64, 64)
batch_size = 128
data_generator = ImageDataGenerator(rescale=1.0 / 255)
train_dataset = data_generator.flow_from_directory('D:\Pycharm\Skin cancer data analysis\Assignment dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode=None,
    shuffle=True
)
generator = make_generator_model()
discriminator = make_discriminator_model()

Found 10015 images belonging to 2 classes.



In [8]:
def train(dataset, epochs):
    max_iterations = 100000
    iteration_count = 0  
    stop_training = False

    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        progress_bar = tqdm(enumerate(train_dataset), total=len(train_dataset), ncols=100)

        for i, image_batch in progress_bar:
            train_step(image_batch)

            iteration_count += 1

            if iteration_count >= max_iterations:
                stop_training = True
                break  # Break out of the inner loop

        if (epoch + 1) % 10 == 0:
            generate_images(generator, tf.random.normal([16, 100]))

        if stop_training:
            break  # Break out of the outer loop
            
        elapsed_time = time.time() - start_time
        remaining_time = timedelta(seconds=(epochs - epoch - 1) * elapsed_time)
        print(f"Elapsed Time: {timedelta(seconds=elapsed_time)}, Remaining Time: {remaining_time}")

        if iteration_count >= max_iterations:
            break  


EPOCHS = 100
train(train_dataset, EPOCHS)

Epoch 1/100


19205it [4:11:19,  1.27it/s]                                                                        


KeyboardInterrupt: 

In [ ]:
generator.compile(optimizer=generator_optimizer, loss=generator_loss)
discriminator.compile(optimizer=discriminator_optimizer, loss=discriminator_loss)

In [ ]:
NOISE_DIM = 100
num_samples = 16

noise = tf.random.normal([num_samples, NOISE_DIM])
generated_images = generator(noise, training=False)
plt.figure(figsize=(8, 8))
for i in range(num_samples):
    plt.subplot(4, 4, i + 1)
    plt.imshow(generated_images[i] * 0.5 + 0.5)
    plt.axis('off')

plt.show()